#01. import libraries

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

#02. Data

In [2]:
path = r'/Users/golde/Instacart Basket Analysis/'

In [3]:
ords_prods_merged =pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_updated.pkl'))

In [5]:
#create subset
df = ords_prods_merged[:1000000]

In [6]:
#agg mean
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,19.765396
7,18.575283
13,18.950838
14,20.460336
16,20.262396
17,12.646785
19,20.168036
20,18.808671


In [7]:
df.groupby('department_id')['order_number'].mean()

department_id
4     19.765396
7     18.575283
13    18.950838
14    20.460336
16    20.262396
17    12.646785
19    20.168036
20    18.808671
Name: order_number, dtype: float64

In [8]:
#agg multiple
df.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
4                19.765396   2  99
7                18.575283   2  99
13               18.950838   2  99
14               20.460336   2  99
16               20.262396   2  99
17               12.646785   2  98
19               20.168036   2  99
20               18.808671   2  99

In [9]:
#agg mean for full dataframe
ords_prods_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,16.559358
2,18.413176
3,18.279600
4,18.915890
5,16.497751
6,17.609390
7,18.303975
8,16.383301


The process took longer to run on the full data frame. It also gives all the department id numbers rather than just a few. 

In [10]:
#creating loyalty flags
ords_prods_merged['max_order'] = ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [11]:
pd.options.display.max_rows = None

In [12]:
ords_prods_merged.loc[ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merged.loc[(ords_prods_merged['max_order'] <= 40) & (ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merged.loc[ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [13]:
ords_prods_merged['loyalty_flag'].value_counts(dropna = False)

Regular customer    15081691
Loyal customer      10095381
New customer         5151691
Name: loyalty_flag, dtype: int64

In [14]:
#check data
ords_prods_merged[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,2
1,1,New customer,3
2,1,New customer,4
3,1,New customer,5
4,1,New customer,6
5,1,New customer,7
6,1,New customer,8
7,1,New customer,9
8,1,New customer,10
9,15,Regular customer,15


In [17]:
#check product price by customer status
ords_prods_merged.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.402162  1.0  99999.0
New customer      13.396333  1.0  99999.0
Regular customer  12.546842  1.0  99999.0

In [20]:
#create spender flag
ords_prods_merged['spender_type'] = ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [21]:
ords_prods_merged.loc[ords_prods_merged['spender_type'] >= 10, 'spender_flag'] = 'High Spender'
ords_prods_merged.loc[ords_prods_merged['spender_type'] < 10, 'spender_flag'] = 'Low Spender'

In [22]:
ords_prods_merged['spender_flag'].value_counts(dropna = False)

Low Spender     29730531
High Spender      598232
Name: spender_flag, dtype: int64

In [27]:
ords_prods_merged[['user_id', 'spender_flag', 'prices']].head(60)

,user_id,spender_flag,prices
0,1,Low Spender,9.0
1,1,Low Spender,9.0
2,1,Low Spender,9.0
3,1,Low Spender,9.0
4,1,Low Spender,9.0
5,1,Low Spender,9.0
6,1,Low Spender,9.0
7,1,Low Spender,9.0
8,1,Low Spender,9.0
9,15,Low Spender,9.0


In [28]:
# create frequency flag
ords_prods_merged['frequency_type'] = ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [29]:
ords_prods_merged.loc[ords_prods_merged['frequency_type'] > 20, 'frequency_flag'] = 'Non-Frequent Customer'
ords_prods_merged.loc[(ords_prods_merged['frequency_type'] <= 20) & (ords_prods_merged['frequency_type'] > 10), 'frequency_flag'] = 'Regular Customer'
ords_prods_merged.loc[ords_prods_merged['frequency_type'] <= 10, 'frequency_flag'] = 'Frequenct Customer'

In [30]:
ords_prods_merged['frequency_flag'].value_counts(dropna = False)

Frequenct Customer       20675290
Regular Customer          6594542
Non-Frequent Customer     3058931
Name: frequency_flag, dtype: int64

In [31]:
ords_prods_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merged_updated2.pkl'))